In [1]:
import pickle
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [2]:
# === LOAD DATA ===
with open("D:\Study\Ai_Music_Composer\project/data/generator_training_data.pkl", "rb") as f:
    data = pickle.load(f)

X_notes = data["X_notes"]
X_moods = data["X_moods"]
y = data["y"]
note_to_idx = data["note_to_idx"]
idx_to_note = data["idx_to_note"]
mood_classes = data["mood_encoder"]

SEQ_LEN = X_notes.shape[1]
vocab_size = len(note_to_idx)
num_moods = X_moods.shape[1]

In [3]:
# === BUILD MODEL ===
note_input = Input(shape=(SEQ_LEN,), name="note_input")
mood_input = Input(shape=(num_moods,), name="mood_input")

embedding = Embedding(input_dim=vocab_size, output_dim=128, input_length=SEQ_LEN)(note_input)
lstm_out = LSTM(256)(embedding)

merged = Concatenate()([lstm_out, mood_input])
dense1 = Dense(256, activation='relu')(merged)
output = Dense(vocab_size, activation='softmax')(dense1)

model = Model(inputs=[note_input, mood_input], outputs=output)
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

model.summary()

C:\Users\OMKAR JHA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ note_input          │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 50, 128)   │     16,384 │ note_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 256)       │    394,240 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mood_input          │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 260)       │          0 │ lstm[0][0],       │
│ (Concatenate)       │                   │            │ mood_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │     66,816 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     32,896 │ dense[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 510,336 (1.95 MB)

 Trainable params: 510,336 (1.95 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
# === TRAIN ===
EPOCHS = 30
BATCH_SIZE = 64

model.fit([X_notes, X_moods], y, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.1)

Epoch 1/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 778s 550ms/step - accuracy: 0.3714 - loss: 2.3670 - val_accuracy: 0.3336 - val_loss: 4.0878
Epoch 2/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 845s 600ms/step - accuracy: 0.7336 - loss: 0.8885 - val_accuracy: 0.2950 - val_loss: 5.1001
Epoch 3/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 832s 591ms/step - accuracy: 0.8547 - loss: 0.4780 - val_accuracy: 0.2944 - val_loss: 5.2272
Epoch 4/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 858s 610ms/step - accuracy: 0.8746 - loss: 0.3984 - val_accuracy: 0.2709 - val_loss: 5.5519
Epoch 5/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 808s 574ms/step - accuracy: 0.8822 - loss: 0.3669 - val_accuracy: 0.3044 - val_loss: 5.8859
Epoch 6/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 669s 475ms/step - accuracy: 0.8853 - loss: 0.3534 - val_accuracy: 0.2911 - val_loss: 5.8890
Epoch 7/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 670s 476ms/step - accuracy: 0.8861 - loss: 0.3429 - val_accuracy: 0.2894 - val_loss: 6.2596
Epoch 8/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 680s 483ms/step - ac

In [5]:
# === SAVE MODEL ===
model.save("D:\Study\Ai_Music_Composer\project\models\music_generator.h5")
print("✅ Generator model trained and saved.")

✅ Generator model trained and saved.
